<a href="https://colab.research.google.com/github/danish330/Royal-Bank-of-Canada-Operational-Risk-Automation-System-/blob/main/Royal_Bank_of_Canada_Operational_Risk_Automation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# =========================================
# RBC Operational Risk Simulation Dashboard
# Author: Danish Aizzat
# =========================================

# Step 1: Setup
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns


# Step 2: Read CSV
csv_url = "https://raw.githubusercontent.com/danish330/Royal-Bank-of-Canada-Operational-Risk-Automation-System-/refs/heads/main/RBC_Trade_Matching_Mock_Data.csv"
df = pd.read_csv(csv_url)


# Step 3: Clean & transform data
df['Match_Time'] = pd.to_datetime(df['Match_Time'], format='%H:%M', errors='coerce').dt.time
df['Match_Hour'] = pd.to_datetime(df['Match_Time'], format='%H:%M:%S', errors='coerce').dt.hour
df['Match_Hour'] = df['Match_Hour'].fillna(-1)

# Simulate EOD trade risk (after 3PM)
df["EOD_Risk"] = np.where(
    (df["Match_Status"] == "Matched Late") & (df["Match_Hour"] >= 15),
    "YES – Escalate", "No"
)

# Simulate Confirmation Tracer
df["Confirmation_Status"] = np.where(
    df["Match_Status"] == "Unmatched", "Pending – Send Tracer", "Confirmed"
)

# Simulate Allocation Status for CTM
df["Allocation_Flag"] = np.where(
    df["CTM_Allocated"] == "No", "Incomplete – Follow Up", "Allocated"
)

# Step 4: KPI Summary
print("\n===== 🧾 RBC Operational KPIs =====")
print(f"Total Trades: {len(df)}")
print(f"Matched: {df[df['Match_Status'] == 'Matched'].shape[0]}")
print(f"Unmatched: {df[df['Match_Status'] == 'Unmatched'].shape[0]}")
print(f"Matched Late: {df[df['Match_Status'] == 'Matched Late'].shape[0]}")
print(f"Late After EOD (3PM): {df[df['EOD_Risk'] == 'YES – Escalate'].shape[0]}")
print(f"CTM Allocated – Yes: {df[df['CTM_Allocated'] == 'Yes'].shape[0]}")
print(f"CTM Allocated – No: {df[df['CTM_Allocated'] == 'No'].shape[0]}")
print(f"Pending Confirmations: {df[df['Confirmation_Status'] == 'Pending – Send Tracer'].shape[0]}")

# Step 5: Match Status Pie Chart
fig1 = px.pie(df, names='Match_Status', title='✅ Trade Match Status')
fig1.show()

# Step 6: Unmatched & Late Trades by Date
unmatched = df[df['Match_Status'] != 'Matched']
trend_data = unmatched.groupby('Trade_Date').size().reset_index(name='Trade_Issues')
fig2 = px.bar(trend_data, x='Trade_Date', y='Trade_Issues', title='📅 Unmatched or Late Trades by Date')
fig2.show()

# Step 7: CTM Allocation Chart
fig3 = px.pie(df, names='CTM_Allocated', title='📤 CTM Allocation Status')
fig3.show()

# Step 8: Trades by Trader & Match Status
fig4 = px.histogram(df, x='Trader_Name', color='Match_Status',
                    barmode='group', title='🧑‍💼 Trades by Trader & Match Status')
fig4.show()

# Step 9: Counterparty Confirmation Analysis
fig5 = px.histogram(df, x='Counterparty', color='Confirmation_Status',
                    barmode='group', title='🏦 Counterparty Confirmation Tracker')
fig5.show()

# Step 10: Risk Escalation Report (Late after 3PM)
print("\n📣 Trades Matched After 3PM (EOD Breach)")
df[df["EOD_Risk"] == "YES – Escalate"][["Trade_ID", "Trade_Date", "Match_Time", "Trader_Name", "Counterparty"]]

# Step 11: Full Operational Trade Table (Audit Log)
print("\n📋 Full Trade Audit Log:")
df_full = df[["Trade_ID", "Trade_Date", "Match_Status", "CTM_Allocated", "Match_Time",
              "EOD_Risk", "Confirmation_Status", "Trader_Name", "Counterparty"]]
df_full.head(20)



===== 🧾 RBC Operational KPIs =====
Total Trades: 100
Matched: 38
Unmatched: 28
Matched Late: 34
Late After EOD (3PM): 11
CTM Allocated – Yes: 51
CTM Allocated – No: 49
Pending Confirmations: 28



📣 Trades Matched After 3PM (EOD Breach)

📋 Full Trade Audit Log:


,Trade_ID,Trade_Date,Match_Status,CTM_Allocated,Match_Time,EOD_Risk,Confirmation_Status,Trader_Name,Counterparty
0,T1000,2022-07-18,Unmatched,Yes,14:06:00,No,Pending – Send Tracer,Aizzat,HSBC
1,T1001,2022-07-06,Matched Late,Yes,10:07:00,No,Confirmed,Ali,Morgan Stanley
2,T1002,2022-07-04,Matched,No,08:17:00,No,Confirmed,Amir,HSBC
3,T1003,2022-07-19,Matched Late,No,12:03:00,No,Confirmed,Ali,Citi
4,T1004,2022-07-29,Unmatched,No,17:31:00,No,Pending – Send Tracer,Nurul,Goldman Sachs
5,T1005,2022-07-26,Matched,Yes,12:26:00,No,Confirmed,Ali,Goldman Sachs
6,T1006,2022-07-19,Matched,Yes,16:22:00,No,Confirmed,Sarah,Citi
7,T1007,2022-07-21,Matched,No,11:11:00,No,Confirmed,Aizzat,Morgan Stanley
8,T1008,2022-07-24,Matched Late,Yes,11:13:00,No,Confirmed,Amir,Goldman Sachs
9,T1009,2022-07-05,Matched,No,11:48:00,No,Confirmed,Nurul,Morgan Stanley
